### Requirement library
<ol>
<li>folium</li>
<li>psycopg2</li>
<li>json</li>
<li>tqdm</li>
</ol>


In [3]:
import folium
# Remove unused imports if not needed
# from folium.utilities import JsCode
# from folium.elements import EventHandler
import psycopg2
import json
from tqdm import tqdm

# Path to the GeoJSON file
geojson_file = '../geodata/properties/geojson/properties_assessment_units.geojson'

def import_geojson(db_conn, geojson_file, table_name):
    """
    Imports GeoJSON data into a PostgreSQL table, displaying a progress bar.

    Args:
        db_conn (psycopg2.connect): A psycopg2 connection object.
        geojson_file (str): The path to the GeoJSON file.
        table_name (str): The name of the PostgreSQL table.
    """
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)
    
    cursor = db_conn.cursor()
    total_features = len(geojson_data['features'])

    with tqdm(total=total_features, desc="Importing GeoJSON") as pbar:
        for feature in geojson_data['features']:
            geometry = feature['geometry']
            properties = feature['properties']

            # Convert geometry to WKT or use ST_GeomFromGeoJSON for GeoJSON
            geom_wkt = json.dumps(geometry)

            # Convert property keys to lowercase (if your table uses lowercase column names)
            properties = {k.lower(): v for k, v in properties.items()}

            # Assuming table has columns matching the properties dict keys
            columns = ', '.join(properties.keys())
            values = ', '.join(['%s'] * len(properties))

            sql = f"INSERT INTO {table_name} (geom, {columns}) VALUES (ST_GeomFromGeoJSON(%s), {values})"
            cursor.execute(sql, (geom_wkt, *properties.values()))

            pbar.update(1)

    cursor.close()
    db_conn.commit()

# Example usage:
try:
    db_conn = psycopg2.connect(
        host="localhost",  # Replace with the IP address or hostname of your Podman container
        port=5432,
        dbname="montreal",  # Replace with the name of your PostgreSQL database
        user="postgres",  # Replace with your PostgreSQL username
        password="postgres"  # Replace with your PostgreSQL password
    )

    print("Connected to PostgreSQL!")

    import_geojson(db_conn, geojson_file, "montreal_properties")
    db_conn.close()
except psycopg2.Error as e:
    print("Error connecting to PostgreSQL:", e)

Connected to PostgreSQL!


Importing GeoJSON: 100%|██████████| 509710/509710 [02:12<00:00, 3848.31it/s]
